<a href="https://colab.research.google.com/github/ulisesjcaroflores/Week8-GSB530F25/blob/main/PA8_1_kNN_and_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Palmer Penguins Modeling

Import the Palmer Penguins dataset and print out the first few rows.

Suppose we want to predict `bill_depth_mm` using the other variables in the dataset.

**Dummify** all variables that require this.

In [1]:
!pip install palmerpenguins

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import *
from palmerpenguins import load_penguins
from sklearn.preprocessing import *
from sklearn.compose import *
import numpy as np

penguins = load_penguins()
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


In [3]:
penguins.isnull().sum()

,0
species,0
island,0
bill_length_mm,2
bill_depth_mm,2
flipper_length_mm,2
body_mass_g,2
sex,11
year,0


In [4]:
penguins = penguins.dropna()

In [5]:
X = penguins.drop(columns='bill_depth_mm')
y = penguins['bill_depth_mm']

In [6]:
penguins.describe() #this is for the numeric vars

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
count,333.000000,333.000000,333.000000,333.000000,333.000000
mean,43.992793,17.164865,200.966967,4207.057057,2008.042042
std,5.468668,1.969235,14.015765,805.215802,0.812944
min,32.100000,13.100000,172.000000,2700.000000,2007.000000
25%,39.500000,15.600000,190.000000,3550.000000,2007.000000
50%,44.500000,17.300000,197.000000,4050.000000,2008.000000
75%,48.600000,18.700000,213.000000,4775.000000,2009.000000
max,59.600000,21.500000,231.000000,6300.000000,2009.000000


Let's use the other variables to predict `bill_depth_mm`. Prepare your data and fit the following models on the entire dataset:

* Your best multiple linear regression model from before
* Two kNN models (for different values of K)
* A decision tree model

Create a plot like the right plot of Fig 1. in our `Model Validation` chapter with the training and test error plotted for each of your four models.

Which of your models was best?

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=321)

#don't use because we need to use grid search to find some models
#just fit on X,y

In [52]:
ct = ColumnTransformer([
    ("scalar", StandardScaler(), make_column_selector(dtype_include=np.number)),
    ("ohe", OneHotEncoder(drop="first", handle_unknown='ignore', sparse_output = False), make_column_selector(dtype_include=object))],
    remainder = 'passthrough'
).set_output(transform="pandas")

scoring = {
    "MSE":  "neg_mean_squared_error",
    "RMSE": "neg_root_mean_squared_error",
    "MAE":  "neg_mean_absolute_error",
    "R2":   "r2",
}

In [53]:
lr_pipeline = Pipeline([
    ("ct", ct),
    ("lr", LinearRegression())
])
lr_pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scalar', StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b04c5a864b0>),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b04c5a850a0>)])),
                ('lr', LinearRegression())])

In [54]:
cv_lr = cross_validate(
    lr_pipeline, X, y,
    scoring=scoring,
    cv=10,
    return_train_score=False
)

result_lr = pd.Series({
    "MSE":  -cv["test_MSE"].mean(),
    "RMSE": -cv["test_RMSE"].mean(),
    "MAE":  -cv["test_MAE"].mean(),
    "R2":   cv["test_R2"].mean(),
},name='Linear Regression')

result_lr

,Linear Regression
MSE,0.720165
RMSE,0.826275
MAE,0.663200
R2,0.491843


In [56]:
#these are the same!

cv_lr = cross_val_score(lr_pipeline, X, y, scoring='neg_mean_squared_error', cv=10)
estimated_mselr = -cv_lr.mean()
print(f"Estimated MSE for Linear Regression: {estimated_mselr:.4f}")

Estimated MSE for Linear Regression: 0.7202


In [12]:
knn_model = KNeighborsRegressor()

pipeline_knn = Pipeline([
    ("ct", ct),
    ("knn", KNeighborsRegressor())
])

In [16]:
paraGrid = {
    'knn__n_neighbors': range(1, 10+1),
    'knn__metric': ['manhattan', 'euclidean'] # Changed 'metric' to 'knn__metric' and standardized metric names
}
search = GridSearchCV(pipeline_knn, paraGrid, cv=10, scoring="neg_mean_squared_error")
KNN_fit = search.fit(X, y)
pd.DataFrame(KNN_fit.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_knn__metric,param_knn__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
8,0.008560,0.000613,0.005649,0.000136,manhattan,9,"{'knn__metric': 'manhattan', 'knn__n_neighbors...",-1.533453,-0.668925,-0.916264,-0.858070,-0.495496,-0.407318,-0.416049,-0.347202,-0.656240,-0.548507,-0.684752,0.335035,1
7,0.009365,0.001956,0.006195,0.000957,manhattan,8,"{'knn__metric': 'manhattan', 'knn__n_neighbors...",-1.536273,-0.688920,-0.966521,-0.788011,-0.513191,-0.421203,-0.405014,-0.349271,-0.650488,-0.532386,-0.685128,0.336042,2
9,0.008468,0.000197,0.005738,0.000157,manhattan,10,"{'knn__metric': 'manhattan', 'knn__n_neighbors...",-1.548268,-0.662568,-0.899938,-0.843770,-0.542006,-0.441315,-0.419539,-0.343661,-0.646394,-0.569627,-0.691709,0.331407,3
16,0.008894,0.000608,0.006043,0.000641,euclidean,7,"{'knn__metric': 'euclidean', 'knn__n_neighbors...",-1.427113,-0.684850,-0.896711,-0.825207,-0.563432,-0.435782,-0.412381,-0.315850,-0.737774,-0.697483,-0.699658,0.300580,4
6,0.008478,0.000164,0.005672,0.000148,manhattan,7,"{'knn__metric': 'manhattan', 'knn__n_neighbors...",-1.527653,-0.752425,-0.997209,-0.768621,-0.515318,-0.414836,-0.410167,-0.355473,-0.739994,-0.533655,-0.701535,0.335883,5
17,0.008675,0.000760,0.005734,0.000413,euclidean,8,"{'knn__metric': 'euclidean', 'knn__n_neighbors...",-1.455437,-0.697096,-0.899053,-0.807652,-0.553144,-0.471477,-0.431430,-0.333679,-0.736340,-0.712566,-0.709787,0.300422,6
5,0.009199,0.000757,0.006050,0.000435,manhattan,6,"{'knn__metric': 'manhattan', 'knn__n_neighbors...",-1.657034,-0.785082,-1.010605,-0.758569,-0.492197,-0.437264,-0.375934,-0.345463,-0.759082,-0.502374,-0.712360,0.375101,7
4,0.008659,0.000464,0.005993,0.000480,manhattan,5,"{'knn__metric': 'manhattan', 'knn__n_neighbors...",-1.734824,-0.790506,-1.007847,-0.705576,-0.486133,-0.410582,-0.413067,-0.344085,-0.734739,-0.496497,-0.712386,0.394138,8
18,0.008886,0.000545,0.005885,0.000244,euclidean,9,"{'knn__metric': 'euclidean', 'knn__n_neighbors...",-1.473057,-0.667495,-0.899325,-0.775799,-0.561302,-0.469959,-0.417662,-0.353872,-0.750138,-0.819592,-0.718820,0.304811,9
15,0.008529,0.000378,0.005744,0.000447,euclidean,6,"{'knn__metric': 'euclidean', 'knn__n_neighbors...",-1.545163,-0.733554,-0.969804,-0.852618,-0.554848,-0.431086,-0.432189,-0.354966,-0.678577,-0.649091,-0.720190,0.330854,10


In [59]:
KNN_fit.best_estimator_.named_steps['knn']

KNeighborsRegressor(metric='manhattan', n_neighbors=9)

In [18]:
#same as the best estimator
KNN_fit.best_params_

{'knn__metric': 'manhattan', 'knn__n_neighbors': 9}

In [65]:
knn_pipe9 = Pipeline([
    ("ct", ct),
    ("knn", KNeighborsRegressor(n_neighbors=7, metric='manhattan'))
])
knn_pipe9.fit(X, y) #find the cv and compare the mse.

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scalar', StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b04c5a864b0>),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b04c5a850a0>)])),
                ('knn',
                 KNeighborsRegressor(metric='manhattan', n_neighbors=7))])

In [74]:
cv_knn9 = cross_validate(
    knn_pipe9, X, y,
    scoring=scoring,
    cv=10,
    return_train_score=False
)

result_knn9 = pd.Series({
    "MSE":  -cv["test_MSE"].mean(),
    "RMSE": -cv["test_RMSE"].mean(),
    "MAE":  -cv["test_MAE"].mean(),
    "R2":   cv["test_R2"].mean(),
},name='KNN Regression (n=9; manhattan)')

result_knn9

,KNN Regression (n=9; manhattan)
MSE,0.720165
RMSE,0.826275
MAE,0.663200
R2,0.491843


In [71]:
cv_knn1 = cross_val_score(knn_pipe9, X, y, scoring='neg_mean_squared_error', cv=5)
estimated_mse2 = -cv_knn1.mean()
print(f"Estimated MSE for KNN Regression with 8 nearest neighbors using manhattan as the metric: {estimated_mse2:.4f}")

Estimated MSE for KNN Regression with 8 nearest neighbors using manhattan as the metric: 0.9164


In [63]:
knn_pipe7 = Pipeline([
    ("ct", ct),
    ("knn", KNeighborsRegressor(n_neighbors=7, metric='euclidean'))
])
knn_pipe7.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scalar', StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b04c5a864b0>),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b04c5a850a0>)])),
                ('knn',
                 KNeighborsRegressor(metric='euclidean', n_neighbors=7))])

In [75]:
cv_knn7 = cross_validate(
    knn_pipe7, X, y,
    scoring=scoring,
    cv=10,
    return_train_score=False
)

result_knn7 = pd.Series({
    "MSE":  -cv["test_MSE"].mean(),
    "RMSE": -cv["test_RMSE"].mean(),
    "MAE":  -cv["test_MAE"].mean(),
    "R2":   cv["test_R2"].mean(),
},name='KNN Regression (n=7; euclidean)')

result_knn7

,KNN Regression (n=7; euclidean)
MSE,0.720165
RMSE,0.826275
MAE,0.663200
R2,0.491843


In [72]:
cv_knn2 = cross_val_score(knn_pipe7, X, y, scoring='neg_mean_squared_error', cv=5)
estimated_mse3 = -cv_knn2.mean()
print(f"Estimated MSE for KNN Regression with 7 nearest neighbors using euclidean as the metric: {estimated_mse3:.4f}")

Estimated MSE for KNN Regression with 7 nearest neighbors using euclidean as the metric: 1.2375


In [ ]:
dt1 = DecisionTreeRegressor(max_depth=2)

dt1

DecisionTreeRegressor(max_depth=2)

# Task
Modify the `paraGrid` dictionary to correctly target the `n_neighbors` parameter of the `KNeighborsRegressor` within the pipeline. Change `'n_neighbors'` to `'knn__n_neighbors'`.

## Adjust `paraGrid` for Pipeline

### Subtask:
Modify the `paraGrid` dictionary to correctly target the `n_neighbors` parameter of the `KNeighborsRegressor` within the pipeline. Change `'n_neighbors'` to `'knn__n_neighbors'`.


## Summary:

### Data Analysis Key Findings
*   The `paraGrid` dictionary was successfully modified to target the `n_neighbors` parameter of the `KNeighborsRegressor` within a pipeline by changing the key from `'n_neighbors'` to `'knn__n_neighbors'`.

### Insights or Next Steps
*   When performing hyperparameter tuning for models within a `Pipeline`, it is crucial to prefix the parameter names with the corresponding step's name followed by a double underscore (e.g., `step_name__parameter_name`) to ensure correct targeting.
